In [84]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, LinearConstraint, Bounds
import yfinance as yf
import matplotlib.pyplot as plt

In [85]:
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "JPM", "XOM", "PG", "V"]

prices = yf.download(tickers, start = '2015-01-01', auto_adjust=False)['Adj Close']
prices.head()


[*********************100%***********************]  10 of 10 completed


Ticker,AAPL,AMZN,GOOGL,JPM,META,MSFT,NVDA,PG,V,XOM
Date,,,,,,,,,,
2015-01-02,24.261053,15.4260,26.319653,46.948071,78.021957,39.998692,0.483066,66.983482,61.462479,58.967144
2015-01-05,23.577570,15.1095,25.818159,45.490582,76.768845,39.630875,0.474906,66.665031,60.105743,57.353691
2015-01-06,23.579794,14.7645,25.180983,44.311043,75.734520,39.049202,0.460508,66.361336,59.718464,57.048763
2015-01-07,23.910429,14.9210,25.106926,44.378670,75.734520,39.545334,0.459308,66.709450,60.518578,57.626823
2015-01-08,24.829126,15.0230,25.194403,45.370373,77.753441,40.708702,0.476586,67.472313,61.330284,58.586010


In [86]:
def expectedreturns(prices, days = 252):
    if not isinstance(prices, (pd.Series, pd.DataFrame)):
        raise TypeError("Format prices as a pd.Series")
    returns = prices.pct_change().dropna()
    exp_ret = returns.mean() * days
    return exp_ret

In [87]:
def covariancematrix(prices, days = 252):
    if not isinstance(prices, (pd.Series, pd.DataFrame)):
        raise TypeError("Formate prices as pd.Series")
    returns = prices.pct_change().dropna()
    covariance = returns.cov() * days

    return covariance

In [96]:
cov_matrix = covariancematrix(prices)
exp_ret = expectedreturns(prices)

/var/folders/f9/1kf93kh165n3n2jnlj8q1f_m0000gn/T/ipykernel_32218/861064831.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()


In [89]:
def minimumvariance(prices, target_return):
    if not isinstance(prices, (pd.Series, pd.DataFrame)):
        raise TypeError("Prices should be a pd.Series")
    
    bounds = Bounds(0,1)
    linear_constraint = LinearConstraint(np.ones(len(tickers)), 1, 1)

    w0 = np.ones(len(tickers)) / len(tickers)
    
    exp_ret = expectedreturns(prices)
    
    cov_matrix = covariancematrix(prices)

    target_constraint = {'type': 'eq', 'fun': lambda w: exp_ret.T @ w - target_return}
    
    variance = lambda w: w.T @ cov_matrix @ w

    res1 = minimize(variance, w0, args = (), method = 'SLSQP', bounds=bounds, constraints=[linear_constraint, target_constraint])
    
    return res1

In [90]:
def efficientfrontier(prices, data_points = 250):
    if not isinstance(prices, (pd.Series, pd.DataFrame)):
        raise TypeError("Prices should be a pd.Series")
    
    exp_ret = expectedreturns(prices)
    cov_matrix = covariancematrix(prices)
    
    bounds = Bounds(0,1)
    linear_constraint = LinearConstraint(np.ones(len(tickers)), 1, 1)

    w0 = np.ones(len(tickers)) / len(tickers)

    ret = lambda w: -exp_ret.T @ w # negative so we can find maximum
    max_ret_port = minimize(ret, w0, args=(), method = 'SLSQP', constraints=linear_constraint, bounds=bounds) 

    var = lambda w : w.T @ cov_matrix @ w
    min_var_port = minimize(var, w0, args = (), method = "SLSQP", constraints=linear_constraint, bounds=bounds)

    min_return = exp_ret.T @ min_var_port.x
    max_return = exp_ret.T @ max_ret_port.x 

    trs = np.linspace(min_return, max_return, data_points, True)

    data = np.zeros((2, data_points))
    weight_list = []

    for tr in trs:
        indx = np.where( trs == tr)
        result = minimumvariance(prices, tr)
        if result.success:
            risk = np.sqrt(result.fun)
            weights = result.x
            weight_list.append(weights)
            returns = np.dot(exp_ret.T, weights)
            data[0, indx] = risk
            data[1, indx] = returns
    
    return data, weight_list


In [91]:
def max_sharpe(prices, data_points = 250):
    if not isinstance(prices, (pd.Series, pd.DataFrame)):
        raise TypeError("Format prices as a pd.Series")
    
    ef = efficientfrontier(prices, data_points)

    ef_data, weights = ef

    sharpe = ef_data[0] / ef_data[1]

    maxsh_idx = np.argmax(sharpe)
    maxsh = sharpe[maxsh_idx]

    return maxsh, weights[maxsh_idx]

In [98]:
a,b = max_sharpe(prices)
b

/var/folders/f9/1kf93kh165n3n2jnlj8q1f_m0000gn/T/ipykernel_32218/2750904288.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()
/var/folders/f9/1kf93kh165n3n2jnlj8q1f_m0000gn/T/ipykernel_32218/2750904288.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()
/var/folders/f9/1kf93kh165n3n2jnlj8q1f_m0000gn/T/ipykernel_32218/2750904288.py:4: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to callin

array([0.00000000e+00, 7.85682501e-02, 5.85255197e-02, 4.32521922e-02,
       5.05702990e-03, 8.02814589e-20, 2.95766423e-20, 5.94138362e-01,
       5.65456460e-02, 1.63913000e-01])